Access the embeddings from kaggle API mentioned in the notrbooks

## Import dependencies

In [1]:
import pandas as pd
import os
import tensorflow as tf
from tensorboard.plugins import projector
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np

2024-06-08 21:37:33.759105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Import data

In [2]:
df = pd.read_csv("infersent_embeddings.csv",nrows=1000,index_col=0) # change the path according to your local
embedding_tensor = tf.convert_to_tensor(df.drop(['prompt'],axis=1))

2024-06-08 21:37:41.581018: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
prompts = df['prompt'].str.replace('\n','').astype(str)

## Plotting using tensorboard

In [7]:
log_dir='logs/embedding/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

#knn
scaler = MinMaxScaler()
normalized_emb =  tf.convert_to_tensor(scaler.fit_transform(embedding_tensor))

kmeans = KMeans(n_clusters=10, random_state=0).fit(normalized_emb)

with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
  f.write("Prompt\tCluster\n")
  for prompt, label in zip(prompts, kmeans.labels_):
    f.write("{}\t{}\n".format(prompt, label))

weights = tf.Variable(embedding_tensor)
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

run ```tensorboard --logdir logs/embedding``` and use the available umap,t-sne,pca vizualizations